In [113]:
!pip install --upgrade prophet

In [104]:
import pandas as pd
import plotly.express as px
from prophet import Prophet
from datetime import datetime


In [94]:
# Step 1: Load data
# Example for loading a CSV file (You can replace with your file path)
df = pd.read_csv('/content/covid_19_clean_complete.csv')
df.head()


,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active,WHO Region
0,NaN,Afghanistan,33.93911,67.709953,2020-01-22,0,0,0,0,Eastern Mediterranean
1,NaN,Albania,41.15330,20.168300,2020-01-22,0,0,0,0,Europe
2,NaN,Algeria,28.03390,1.659600,2020-01-22,0,0,0,0,Africa
3,NaN,Andorra,42.50630,1.521800,2020-01-22,0,0,0,0,Europe
4,NaN,Angola,-11.20270,17.873900,2020-01-22,0,0,0,0,Africa


In [118]:
# Convert the 'Date' column to datetime format
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')

In [96]:
# Check if there are any missing values after conversion
print(df['Date'].isnull().sum())

# If there are any NaT values (Not a Time) in the 'Date' column, you can filter those out
df = df.dropna(subset=['Date'])


0


In [97]:
# Filter data for a specific country/region, e.g., 'World' (adjust if necessary)
df_world = df.groupby('Date').agg({
    'Confirmed': 'sum',
    'Deaths': 'sum',
    'Recovered': 'sum',
    'Active': 'sum'
}).reset_index()


In [119]:
# Step 1: Data Preparation
df_confirmed = df_world[['Date', 'Confirmed']]  # Ensure the column names are correct
df_confirmed.columns = ['ds', 'y']  # Rename columns for Prophet


In [120]:
# Ensure 'ds' column is of datetime type
df_confirmed['ds'] = pd.to_datetime(df_confirmed['ds'])

# Step 2: Initialize the Prophet Model
model = Prophet()

# Step 3: Fit the model with the existing data
model.fit(df_confirmed)

<ipython-input-120-0a566cad3750>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpqhk5tnnf/j32pw8hf.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpqhk5tnnf/p2y2fmte.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=24862', 'data', 'file=/tmp/tmpqhk5tnnf/j32pw8hf.json', 'init=/tmp/tmpqhk5tnnf/p2y2fmte.json', 'output', 'file=/tmp/tmpqhk5tnnf/prophet_modeli6edw4

In [126]:
# Step 7: Create a dataframe with future dates for prediction
# Ensure 'periods' and 'freq' are passed correctly to avoid conflicts
future = model.make_future_dataframe(df_confirmed, periods=7, freq='D')  # Predict for the next 7 days

# Fit the model with the data
model.fit(df_confirmed)

# Step 8: Make predictions
forecast = model.predict(future)

# Plot the forecast
model.plot(forecast)

TypeError: Prophet.make_future_dataframe() got multiple values for argument 'periods'

In [128]:
df_confirmed.head()  # Check if dataframe is in the right format


,ds,y
0,2020-01-22,555
1,2020-01-23,654
2,2020-01-24,941
3,2020-01-25,1434
4,2020-01-26,2118
